## Pre-processing the data
Code copied from Starter_Code.ipynb

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dla-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(['EIN', 'NAME'], axis=1, inplace=True)

In [3]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`

# Set a cutoff value
cutoff_value = 500
#  Filter the value counts to get only those below the cutoff
below_cutoff = application_df['APPLICATION_TYPE'].value_counts()[application_df['APPLICATION_TYPE'].value_counts() < cutoff_value]
# Create the list of `application_types_to_replace`
application_types_to_replace = below_cutoff.index.tolist()

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [4]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
# Set a cutoff value
cutoff_value = 1500
#  Filter the value counts to get only those below the cutoff
below_cutoff = application_df['CLASSIFICATION'].value_counts()[application_df['CLASSIFICATION'].value_counts() < cutoff_value]
# Create the list of `classifications_to_replace`
classifications_to_replace = below_cutoff.index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [5]:
# make a copy of dataframe to test for dropping columns to acheive >75% accuracy
application_dropCol_df = application_df.copy()


In [6]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df, columns=['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE', 'ORGANIZATION', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS'])
application_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
1,1,108590,1,False,False,False,True,False,False,False,...,True,False,False,False,False,False,False,False,True,False
2,1,5000,0,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,True,False
3,1,6692,1,False,False,False,True,False,False,False,...,False,True,False,False,False,False,False,False,True,False
4,1,142590,1,False,False,False,True,False,False,False,...,False,False,True,False,False,False,False,False,True,False


## Create a new neural network model
Implementing at least three model optimisation methods

## Method 1 Adjust the input data by dropping some columns

In [7]:
 # Drop the 'ORGANIZATION' and 'SPECIAL_CONSIDERATIONS' columns from the dataframe
application_dropCol_df.drop(['ORGANIZATION', 'SPECIAL_CONSIDERATIONS'], axis=1, inplace=True)
application_dropCol_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,STATUS,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,1,0,5000,1
1,T3,Independent,C2000,Preservation,1,1-9999,108590,1
2,T5,CompanySponsored,C3000,ProductDev,1,0,5000,0
3,T3,CompanySponsored,C2000,Preservation,1,10000-24999,6692,1
4,T3,Independent,C1000,Heathcare,1,100000-499999,142590,1


In [8]:
# Convert categorical data to numeric with `pd.get_dummies`
application_dropCol_df = pd.get_dummies(application_dropCol_df, columns=['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE', 'INCOME_AMT'])
application_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
1,1,108590,1,False,False,False,True,False,False,False,...,True,False,False,False,False,False,False,False,True,False
2,1,5000,0,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,True,False
3,1,6692,1,False,False,False,True,False,False,False,...,False,True,False,False,False,False,False,False,True,False
4,1,142590,1,False,False,False,True,False,False,False,...,False,False,True,False,False,False,False,False,True,False


In [9]:
# Split our preprocessed data into our features and target arrays
# create the target array
target = application_dropCol_df['IS_SUCCESSFUL'].values
# Drop the target variable from the DataFrame to obtain the features
features = application_dropCol_df.drop('IS_SUCCESSFUL', axis=1)
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(features, target, random_state=78)

In [10]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [11]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = X_train.shape[1]
hidden_nodes_layer1 =  80
hidden_nodes_layer2 = 30

nn1 = tf.keras.models.Sequential()

In [12]:
# First hidden layer
nn1.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn1.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn1.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                3040      
                                                                 
 dense_1 (Dense)             (None, 30)                2430      
                                                                 
 dense_2 (Dense)             (None, 1)                 31        
                                                                 
Total params: 5501 (21.49 KB)
Trainable params: 5501 (21.49 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [13]:
# Compile the model
nn1.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
# Train the model use the callback
fit_model = nn1.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 7s 6ms/step - loss: 0.5785 - accuracy: 0.7174
Epoch 2/100
804/804 [==============================] - 7s 9ms/step - loss: 0.5638 - accuracy: 0.7256
Epoch 3/100
804/804 [==============================] - 4s 5ms/step - loss: 0.5626 - accuracy: 0.7266
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5608 - accuracy: 0.7277
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5602 - accuracy: 0.7281
Epoch 6/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5594 - accuracy: 0.7290
Epoch 7/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5586 - accuracy: 0.7284
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5587 - accuracy: 0.7282
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5577 - accuracy: 0.7288
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5579 - accura

In [14]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn1.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5639 - accuracy: 0.7235 - 500ms/epoch - 2ms/step
Loss: 0.5639365911483765, Accuracy: 0.723498523235321


In [15]:
# Export our model to HDF5 file
# Define the file path for saving the model
# saving to temp storage for evaluation
filepath = "AlphabetSoupCharity_M1.h5"

# Save the model to HDF5 format
nn1.save(filepath)

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


## Method 2 - Use more neurons for the hidden layers

> Add blockquote



In [16]:
# Split our preprocessed data into our features and target arrays
# create the target array
y = application_df['IS_SUCCESSFUL'].values
# Drop the target variable from the DataFrame to obtain the features
X = application_df.drop('IS_SUCCESSFUL', axis=1)
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [17]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [18]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = X_train.shape[1]
hidden_nodes_layer1 =  128
hidden_nodes_layer2 = 60

nn2 = tf.keras.models.Sequential()

In [19]:
# First hidden layer
nn2.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn2.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 128)               5632      
                                                                 
 dense_4 (Dense)             (None, 60)                7740      
                                                                 
 dense_5 (Dense)             (None, 1)                 61        
                                                                 
Total params: 13433 (52.47 KB)
Trainable params: 13433 (52.47 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [20]:
# Compile the model
nn2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
# Train the model use the callback
fit_model = nn2.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 3s 2ms/step - loss: 0.5721 - accuracy: 0.7243
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5579 - accuracy: 0.7302
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5534 - accuracy: 0.7306
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5513 - accuracy: 0.7330
Epoch 5/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5501 - accuracy: 0.7320
Epoch 6/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5491 - accuracy: 0.7328
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5486 - accuracy: 0.7327
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5481 - accuracy: 0.7324
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5470 - accuracy: 0.7340
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5467 - accura

In [21]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5591 - accuracy: 0.7261 - 465ms/epoch - 2ms/step
Loss: 0.5590918064117432, Accuracy: 0.726064145565033


In [22]:
# Export our model to HDF5 file
# Define the file path for saving the model
# saving to temp storage for evaluation
filepath = "AlphabetSoupCharity_M2.h5"

# Save the model to HDF5 format
nn2.save(filepath)

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


## Method 3 - Add more hidden layers - increase hidden layers to 4.

In [23]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = X_train.shape[1]
hidden_nodes_layer1 =  128
hidden_nodes_layer2 = 64
hidden_nodes_layer3 = 30

nn3 = tf.keras.models.Sequential()

In [25]:
# First hidden layer
nn3.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)
# Second hidden layer
nn3.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))
# Third hidden layer
nn3.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))
# Output layer
nn3.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))


# Check the structure of the model
nn3.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 128)               5632      
                                                                 
 dense_7 (Dense)             (None, 64)                8256      
                                                                 
 dense_8 (Dense)             (None, 128)               8320      
                                                                 
 dense_9 (Dense)             (None, 64)                8256      
                                                                 
 dense_10 (Dense)            (None, 30)                1950      
                                                                 
 dense_11 (Dense)            (None, 1)                 31        
                                                                 
Total params: 32445 (126.74 KB)
Trainable params: 3244

In [ ]:
# Compile the model
nn3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
# Train the model use the callback
fit_model = nn3.fit(X_train_scaled,y_train,epochs=100)

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn3.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
# Export our model to HDF5 file
# Define the file path for saving the model
filepath = "AlphabetSoupCharity_M3.h5"

# Save the model to HDF5 format
nn3.save(filepath)

## Evaluate the models and select the best one.
Save as AlphabetSoupCharity_Optimisation.h5

Original Method accuracy 72.51% loss 56.21%

Method 1 accuracy 72.35% loss 56.46%

Method 2 accuracy 72.57% loss 55.93

Method 3 accuracy 72.61% loss 56.59%

In [ ]:
# Activate google drive path
from tensorflow.keras.models import load_model

# Specify the filepath of the best model
filepath = "AlphabetSoupCharity_M3.h5"
# Load the model from the HDF5 file
loaded_model = load_model(filepath)

In [ ]:
# Export our model to HDF5 file
# Mount the google drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# specify the filepath
filepath = "/content/drive/My Drive/UWA Bootcamp/AlphabetSoupCharity_Optimisation.h5"
loaded_model.save(filepath)